### 01 - Python Fundamentals Part 1

#### Outline

* Imports & Libraries
* Variables, Primitives, Arithmetic, and String Formatting
* Example: Building Machinery
* Example: Making Plots
* Strings, Paths, and Navigation

#### Commentary

This class is going to provide _exposure_ to a wide range of concepts and a set of recipes and best-practices to lean on. It's also taking a depth-first approach: rather than show several ways to do something and discuss tradeoffs, we'll show one good way, show any neighboring landmines, and move on.

Concrete skill takes long exposure that simply isn't practical to build in one hour per week, so instead, the focus will be on lowering barriers and greasing the rails for organic learning outside.

Motivation: exponentiation, project transitions, reproducibility

----

#### Imports and Libraries

Unlike MATLAB, Python requires explicitly importing components, including most of the standard library. This may feel inconvenient at first, but it's one of the things that allows building structured and repeatable code.

In [ ]:
# These are part of the standard library, but we have to import them anyway
import os
import math

# This is an implied "relative import" from the same folder
# An explicit relative import would be like "from . import example_module,"
# but that only works inside a proper package.
from example_module import HELLO_CONSTANT

print("The working folder is ", os.path.abspath(''))
print("We can add some numbers: ", math.fsum([1.0, 2.0, 3.0]))
print("Matplotlib knows we're in a notebook: ", os.environ.get("MPLBACKEND"))
print("We can import our own stuff, too: ", HELLO_CONSTANT)

----

#### Variables, Primitives, and Arithmetic

Unlike MATLAB, but similar to other programming languages, Python has multiple kinds of numbers in addition to other data types.

Discussion: type annotation, O-notation, pass-by-assignment

In [ ]:
import numpy as np  # Dense arrays
from scipy import sparse  # Sparse matrices
from pint import Quantity, UnitRegistry  # Unit conversions

# Integers can be written in base 10, 16, or 2.
# Usually only base 10 is needed for engineering outside embedded systems or graphics.
#
# Python 3 integers are not a true primitive type; each one is an object that takes up at least 28 bytes,
# can extend to arbitrary size to accommodate large integers, and will automatically become a float
# when needed to represent non-integer values.
a: int = 5
a_hex: int = 0x05  # Hexadecimal representation
a_bin: int = 0b0101  # Binary
a = 6  # Mutate the existing value of `a`

# Floats can be written as either decimal or scientific notation.
# Python floats are 64-bit by default.
b: float = 1.0
b_sci: float = 10.0e-1
c: complex = complex(a, b)  # Nobody uses this

# Booleans represent logical operations.
a: bool = True  # Shadowing `a`
yep: bool = True
nope: bool = False
sure: bool = a > 3  # Comparison operations produce a boolean output

# Nonetype is special, and usually represents a missing value
something_should_be_here: float | None = None

# Strings come in a few flavors. You can also change the encoding if needed.
d: str = "yeet"
d_bytes: bytes = b"yeet"  # Byte string
d_format: str = f"asdf: {d}"  # Trivial use of format string or "f-string"
d_format_2: str = "asdf: {}".format(d)  # Old syntax
d_literal: str = r"\asdf"  # String literal - mostly used for formatting LaTeX

# Tuples are like lists with a specific length and specific type in each index.
# They just glue some variables together:
e: tuple[int, float] = (a, b)
ee: tuple[int, float, str] = (a, b, d)
e[0]
e[1]

# Lists _can_ contain different types, but best practice is to avoid this
bad_list: list[str | float] = ["asdf", 3.1]  # Don't do this unless you really need it
good_list: list[float] = [1.0, 2.0, 3.0]
two = good_list[0]  # List indexing is O(1), indexed from 0

# Dictionaries allow looking a value up by name
stuff: dict[str, str] = {
    "foo": "bar",
    "bar": "baz"
}
bar = stuff["foo"]  # Dictionary access is O(1)

# Dense arrays can also be used as matrices.
# Matlab internally represents most matrices as sparse (specifically compressed column).
# In Python and other languages, arrays and matrices come in different varieties.
dense_array: np.typing.NDArray = np.array([1.0, 3.0, 2.0])
dense_mat: np.typing.NDArray = np.diag(dense_array)
sparse_mat: sparse.csc_matrix = sparse.csc_matrix(dense_mat)  # Not the most efficient way to do this

# NOTE: Discuss dense matrix memory format
# [ 1, 0, 0 ]
# [ 0, 3, 0 ]
# [ 0, 0, 2 ]

# Values can have associated units and do conversions automatically
ureg = UnitRegistry(system="mks")  # Setup
Q_ = ureg.Quantity  # More setup

array_with_units: Quantity = Q_(dense_array, "kg * m / s^2")
scalar_with_units: Quantity = Q_(9.805, "ft / fortnight / slug") # I don't want to deal with this conversion!
result_with_units: Quantity = array_with_units / scalar_with_units  # Units tracked, but no conversions applied yet
result_with_reduced_units: Quantity = result_with_units.to_base_units()  # Applied conversions

print(result_with_reduced_units)


In [ ]:
# Format strings can format almost anything

print(f"Format some numbers: {a}, {b}")
print(f"Format a list: {bad_list}")
print(f"Format a dictionary: {stuff}")

In [ ]:
# Common arithmetic operations are available as operators.
# Some notables:
a_squared = a ** 2.1  # Exponentiation is different from matlab
a += 1
a -= 1
dense_mat @ dense_array  # Matrix multiplication is explicit, unlike matlab
sparse_mat @ dense_array
solved = sparse.linalg.factorized(sparse_mat)(dense_array)  # Matlab backslash operator - solve a system

# Python automatically typecasts integers to floats for arithmetic that can produce fractions,
# even if they don't in a given particular case.
# This is called "implicit typecasting."
int_times_int = 2 * 3
int_over_float = 1 / 3.0
int_over_int_fractional = 1 / 2
int_over_int_exact = 4 / 2

print("int_times_int ->", type(int_times_int))
print("int_over_float ->", type(int_over_float))
print("int_over_int_fractional ->", type(int_over_int_fractional))
print("int_over_int_exact ->", type(int_over_int_exact))

##### Example: Building Machinery

In [ ]:
# Don't find the thing you want? You can build it!

# This very small block adds a wrapper for scipy matrices that allows solving systems like
# Ax = b -> x = A // b

from functools import cached_property

class MatrixWithSolver(sparse.csc_matrix):
    """A sparse matrix with shorthand for running a fast direct solver like `x = A // b`."""
    
    @cached_property
    def solve(self):
        """
        Compute LU factorization. Only runs once, then is stored and reused.
        Actual procedure defers to SuiteSparse UMFPACK direct solver.
        """
        return sparse.linalg.factorized(self)

    def __floordiv__(self, other: np.typing.NDArray) -> np.typing.NDArray:
        """
        Solve a linear system with the provided RHS.
        Uses pre-computed LU factorization and stored symbolic solution.
        """
        return self.solve(other)

sparse_mat_wrapped = MatrixWithSolver(sparse_mat)
solved_shorthand = sparse_mat_wrapped // dense_array

assert np.all(solved == solved_shorthand)

##### Example: Making Plots

The `matplotlib` library provides a plotting interface very similar to matlab. There are other plotting libraries like `plotly` that will be covered later on.

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2)

x = np.linspace(0, 2.0 * np.pi, 20)

plt.sca(axes[0])
y1 = np.sin(x)
plt.plot(x, y1, label="sine")
plt.legend()

y2 = np.tan(x)
plt.sca(axes[1])
plt.plot(x, y2, label="tan")
plt.legend()

plt.suptitle(r"Plots! $y_{1} = sin(\theta)$, $y_{2} = tan(\theta)$")

##### Floats vs. Ints

Floating-point and integer numbers are the most common kinds.

There are others - fixed-point, posits, quaternions, and many different sizes of float and int that have their uses, but aren't the focus here.

The bottom line is that floats have _relative_ resolution (about 2.2e-16 for f64 and about 1.2e-7 for f32), and integers have _absolute_ resolution (1 because that's what an integer is). Fixed-point numbers represent decimal values like a float, but have a fixed absolute resolution like an int; they are very fast and reliable, but not accurate enough for scientific computing.

Both floats and ints have a minimum and maximum value that they can represent.

Some reading material:
* There is a plot in here a couple pages down showing relative resolution for 32-bit floats https://spectrum.ieee.org/floating-point-numbers-posits-processor
* Rust docs for related constants, like the absolute resolution of a 64-bit float near 1.0 (epsilon) https://doc.rust-lang.org/std/primitive.f64.html#associatedconstant.EPSILON
* The fsum full-precision summation procedure gives a quick window into deliberate handling of roundoff error https://docs.python.org/3/library/math.html#math.fsum
    * The original paper is from Jonathan Shewchuck's work on full-precision computational geometry that became the modern field of meshing

<img src="static/01/image.png" width="500"/>

----

#### Strings, Paths, and Navigation

Strings are often used to describe filenames or relative paths where they are provided as user inputs. Best practice is to convert them to Path objects internally, which provides both convenience and cross-platform compatibility.

In [ ]:
# These are part of the standard library
import os
from pathlib import Path

In [ ]:
# Getting the working directory in a notebook environment is different from elsewhere
working_directory = Path(os.path.abspath(''))  # Immediately stuff the string into a Path object

# Outside a notebook, we'd do something like
# here = Path(__file__).parent
# wdir = Path(".").parent

print("You are here: ", working_directory)

In [ ]:
# We can list files/folders
stuff_in_workdir = list(working_directory.iterdir())
stuff_in_workdir

In [ ]:
# We can search for names
markdown_files = list(working_directory.glob("*.md"))
markdown_files

In [ ]:
# Navigating downward
# "./static/01"
# "~/git" -> "/home/jlogan/git"
static_dir = working_directory / "static" / "01"  # {here}/static/01
old_method_static_dir = os.path.join(working_directory, "static", "01")  # Don't need this anymore

# Navigating upward
parent_dir = working_directory.parent
parent_dir_alternate = working_directory / "../"
assert parent_dir.resolve() == parent_dir_alternate.resolve()